In [1]:
from collections import defaultdict

from pathlib import Path
import os
import sys
import json
import pickle
#os.environ['CUDA_VISIBLE_DEVICES']='0'  # don't occupy GPU, inference can work on CPU
import torch
import logging
import numpy as np
from tqdm import tqdm
import pandas as pd
from typing import List, Dict, Tuple

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from flair.embeddings import BertEmbeddings
from flair.data import Sentence
from sklearn.pipeline import Pipeline

from python_ev.dataset import load_parts, load_df
from vectorizers.bert_boris.bert_base_spwsi import BertBaseSPWSI
from subtask1_solutions.base_solution import Subtask1BaseVectorizerFactory
import flair

02/17/2019 11:50:16 - INFO - summarizer.preprocessing.cleaner -   'pattern' package not found; tag filters are not available for English
/home/vladimir/anaconda3/envs/dialogue2019gapping/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
from vectorizers.bert_boris.bert_flair_from_notebook_vectorizer import BertFlairFromNotebookVectorizer, test

vectorizer = BertFlairFromNotebookVectorizer(tokens_field='FORM',
                                             model='bert-base-multilingual-cased', 
                                             add_bias=True, 
                                             normalize_out_embs=True, 
                                             pattern='_',
                                             prediction_cutoff=20,
                                             lemmatize_type=None,
                                             verbose=True, 
                                             exclude_lemma=False)

Loading bert pretrained: bert-base-multilingual-cased
Using torch device:  cuda
Loading bert done
Hyperparameters:
Add_bias: True
Normalize_out_embs: True
Pattern: _



In [3]:
sent = 'Ей он рассказывает одно, а нам - ток совершенно другое'.split(' ')
key = vectorizer.get_inst_id(7, 'dsdsg')
dct = {key: (sent, 7, '')}

In [4]:
vectorizer.predict_sent_substitute_representatives(dct)

Loading vocabulary from pretrained: bert-base-multilingual-cased


generate_vecs: 0it [00:00, ?it/s]
generate_vecs: 1it [00:00,  5.17it/s]00:00<?, ?it/s]
Vectorization: 100%|██████████| 1/1 [00:00<00:00,  5.18it/s]

predict_word_scores:   0%|          | 0/1 [00:00<?, ?it/s]

Loading vocabulary done
INPUT TO BERT: [CLS] | Е | ##й | он | рассказывает | одно | , | а | нам | - | >>[MASK]<< | совершенно | другое | [SEP]


predict_word_scores: 100%|██████████| 1/1 [00:00<00:00,  5.21it/s]


{'7.dsdsg': ['то и это - в он на во о что рассказывает есть ему , не говорит как только за его']}

In [5]:
def gen_list_of_inputs(sent, tok = 'MASK', lemma = ''):
    tokens = sent.split(' ')
    res = []
    for i in range(0, len(tokens) + 1):
        tokens = sent.split(' ')
        tokens.insert(i, tok)
        tokens.insert(i+1, tok) # убрать
        res.append({str(i):(tokens, i, lemma)})
    return res

In [6]:
#sent = 'Акциз на бензин в США при розничных продажах составляет 10 центов за литр, на дизельное топливо - 12 центов.'
#sent = 'Я думаю, она вскоре будет на ногах и через несколько дней на пути к выздоровлению.'
#sent = 'Работа с двухбайтовыми наборами символов — просто кошмар для программиста, так как часть их состоит из одного байта, а часть — из двух.'
sent = 'По состоянию на 31 декабря 2010 года продажи алмазов сторонам, контролируемым государством, включали продажи Государственному фонду драгоценных металлов и драгоценных камней Российской Федерации в размере 25 млн. руб. (по состоянию на 31 декабря 2009 года - 27 060 млн. руб.).'
key = 'sent.sent'

bert_inps = gen_list_of_inputs(sent)

In [7]:
bert_inps

[{'0': (['MASK',
    'MASK',
    'По',
    'состоянию',
    'на',
    '31',
    'декабря',
    '2010',
    'года',
    'продажи',
    'алмазов',
    'сторонам,',
    'контролируемым',
    'государством,',
    'включали',
    'продажи',
    'Государственному',
    'фонду',
    'драгоценных',
    'металлов',
    'и',
    'драгоценных',
    'камней',
    'Российской',
    'Федерации',
    'в',
    'размере',
    '25',
    'млн.',
    'руб.',
    '(по',
    'состоянию',
    'на',
    '31',
    'декабря',
    '2009',
    'года',
    '-',
    '27',
    '060',
    'млн.',
    'руб.).'],
   0,
   '')},
 {'1': (['По',
    'MASK',
    'MASK',
    'состоянию',
    'на',
    '31',
    'декабря',
    '2010',
    'года',
    'продажи',
    'алмазов',
    'сторонам,',
    'контролируемым',
    'государством,',
    'включали',
    'продажи',
    'Государственному',
    'фонду',
    'драгоценных',
    'металлов',
    'и',
    'драгоценных',
    'камней',
    'Российской',
    'Федерации',
    'в',
    

In [8]:
vectorizer = BertFlairFromNotebookVectorizer(tokens_field='FORM',
                                             model='bert-base-multilingual-cased', 
                                             add_bias=True, 
                                             normalize_out_embs=True, 
                                             pattern='_',
                                             prediction_cutoff=20,
                                             lemmatize_type=None,
                                             verbose=True, 
                                             exclude_lemma=False)

results = []
for bert_inp in bert_inps[:1]:
    res = vectorizer.predict_sent_substitute_representatives(bert_inp)
    results.append((list(bert_inp.values())[0][0], list(res.values())[0]))

generate_vecs: 0it [00:00, ?it/s]
Vectorization:   0%|          | 0/1 [00:00<?, ?it/s]
generate_vecs: 1it [00:00, 37.54it/s]00:00<00:00, 47.63it/s]
predict_word_scores: 100%|██████████| 1/1 [00:00<00:00,  6.82it/s]

Hyperparameters:
Add_bias: True
Normalize_out_embs: True
Pattern: _

INPUT TO BERT: [CLS] | >>[MASK]<< | MA | ##S | ##K | По | состоянию | на | 31 | декабря | 2010 | года | про | ##даж | ##и | ал | ##ма | ##зов | сторона | ##м | , | кон | ##тро | ##ли | ##ру | ##ем | ##ым | государство | ##м | , | в | ##кл | ##ю | ##чали | про | ##даж | ##и | Г | ##ос | ##уда | ##рст | ##вен | ##ному | фонду | др | ##аг | ##о | ##цен | ##ных | металл | ##ов | и | др | ##аг | ##о | ##цен | ##ных | кам | ##ней | Российской | Федерации | в | размер | ##е | 25 | млн | . | р | ##уб | . | ( | по | состоянию | на | 31 | декабря | 2009 | года | - | 27 | 060 | млн | . | р | ##уб | . | ) | . | [SEP]


In [9]:
results

[(['MASK',
   'MASK',
   'По',
   'состоянию',
   'на',
   '31',
   'декабря',
   '2010',
   'года',
   'продажи',
   'алмазов',
   'сторонам,',
   'контролируемым',
   'государством,',
   'включали',
   'продажи',
   'Государственному',
   'фонду',
   'драгоценных',
   'металлов',
   'и',
   'драгоценных',
   'камней',
   'Российской',
   'Федерации',
   'в',
   'размере',
   '25',
   'млн.',
   'руб.',
   '(по',
   'состоянию',
   'на',
   '31',
   'декабря',
   '2009',
   'года',
   '-',
   '27',
   '060',
   'млн.',
   'руб.).'],
  ['- : . 1 2 3 5 4 ; , С * [UNK] + • 12 « В H 10'])]

In [10]:
top_elems = 5

for elem in results: # ([elems], [str])
    fst = elem[0]
    snd = elem[1][0].split(' ')[:top_elems]
    snd = [elem for elem in snd if elem.isalpha()]
    if (not set(fst).isdisjoint(snd)):
        print(fst)
        print(snd)
        print('------------')

In [11]:
vectorizer.tokenizer.tokenize('элементарных')

['элемент', '##ар', '##ных']

In [12]:
vectorizer.predict_sent_substitute_representatives(bert_inp)

generate_vecs: 0it [00:00, ?it/s]
Vectorization:   0%|          | 0/1 [00:00<?, ?it/s]
generate_vecs: 1it [00:00, 25.96it/s]00:00<00:00, 29.05it/s]
predict_word_scores:   0%|          | 0/1 [00:00<?, ?it/s]

INPUT TO BERT: [CLS] | >>[MASK]<< | MA | ##S | ##K | По | состоянию | на | 31 | декабря | 2010 | года | про | ##даж | ##и | ал | ##ма | ##зов | сторона | ##м | , | кон | ##тро | ##ли | ##ру | ##ем | ##ым | государство | ##м | , | в | ##кл | ##ю | ##чали | про | ##даж | ##и | Г | ##ос | ##уда | ##рст | ##вен | ##ному | фонду | др | ##аг | ##о | ##цен | ##ных | металл | ##ов | и | др | ##аг | ##о | ##цен | ##ных | кам | ##ней | Российской | Федерации | в | размер | ##е | 25 | млн | . | р | ##уб | . | ( | по | состоянию | на | 31 | декабря | 2009 | года | - | 27 | 060 | млн | . | р | ##уб | . | ) | . | [SEP]


predict_word_scores: 100%|██████████| 1/1 [00:00<00:00,  5.81it/s]


{'0': ['- : . 1 2 3 5 4 ; , С * [UNK] + • 12 « В H 10']}

In [13]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [14]:
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
      

In [15]:
text = "Я думаю, она вскоре будет на ногах и через несколько дней на пути к выздоровлению."

In [16]:
tok = tokenizer.tokenize(text)

In [17]:
tok

['я',
 'ду',
 '##ма',
 '##ю',
 ',',
 'она',
 'вскоре',
 'будет',
 'на',
 'но',
 '##гах',
 'и',
 'через',
 'несколько',
 'д',
 '##не',
 '##и',
 'на',
 'пути',
 'к',
 'вы',
 '##здо',
 '##ров',
 '##лению',
 '.']

In [18]:
tok.insert(17, '[MASK]')

In [19]:
tok.insert(0, '[MASK]')

In [20]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tok)

In [21]:
segments_ids = [0 for x in range(len(indexed_tokens))]

In [22]:
tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')
segments_tensors = torch.tensor([segments_ids]).to('cuda')
model.to('cuda')

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediate(
      

In [23]:
segments_tensors

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], device='cuda:0')

In [24]:
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

In [25]:
predicted_index = torch.argmax(predictions[0, 17]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

In [26]:
predicted_token

'##и'

In [176]:
def bottom_layer(tok_sent, verb_list, tokenizer, index, model, threshold):
    # получаем предложение, список глаголов, токенизатор, индекс конца слова; возвращаем "похожие" токенизированные слова
    # также подаём на вход модель; threshold - количество получаемых элементов
    model.to('cuda')
    #tok_sent = tokenizer.tokenize(sent) # пока что
    res = {}
    if (len(verb_list) == 1):
        verb = verb_list[0] # список из k элементов (k - количество токенов)
        num_tokens = len(verb)
        for i in range(index+2, len(tok_sent)+1):
            if (i == len(tok_sent) or tok_sent[i].isalpha() or tok_sent[i].isdigit()):
                new_tok = tok_sent.copy()
                print(tok_sent[i-1])
                print('=====')
                # тогда перед этим всем вставляем num_tokens токенов
                for _ in range(num_tokens):
                    new_tok.insert(i, '[MASK]')
                print(f'INPUT_TO_BERT: {new_tok}')
                indexed_tokens = torch.tensor([tokenizer.convert_tokens_to_ids(new_tok)]).to('cuda')
                segment_ids = torch.tensor([[0 for x in range(indexed_tokens.shape[1])]]).to('cuda')
                #print(indexed_tokens)
                #print(segment_ids)
                with torch.no_grad():
                    predictions = model(indexed_tokens, segment_ids)
                predictions = predictions.cpu()
                res[i] = []
                for j in range(i, i+num_tokens):
                    #predictions.cpu()[0, j].numpy().argsort(axis = 1).T[-10:][::-1].T
                    predicted_indexes = predictions.cpu()[0, j].numpy().argsort()[-threshold:][::-1]
                    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indexes)
                    res[i].append(predicted_tokens)
    return res

In [28]:
st = 'Магазинчик был маленьким, а мясник и его супруга очень большими, поэтому, когда к ним прибавились мы с женой, в лавке стало совсем тесно.'
tokenizer.tokenize(st)
#tokenizer.tokenize('содержат')

['магазин',
 '##чик',
 'был',
 'мал',
 '##ень',
 '##ким',
 ',',
 'а',
 'м',
 '##яс',
 '##ник',
 'и',
 'его',
 'су',
 '##пруга',
 'очень',
 'большим',
 '##и',
 ',',
 'поэтому',
 ',',
 'когда',
 'к',
 'ним',
 'при',
 '##ба',
 '##вили',
 '##сь',
 'мы',
 'с',
 'же',
 '##но',
 '##и',
 ',',
 'в',
 'ла',
 '##в',
 '##ке',
 'стало',
 'сов',
 '##сем',
 'те',
 '##сно',
 '.']

In [29]:
bottom_layer(st, [[' ' for x in range(1)]], tokenizer, 0, model, 10)

##чик
=====
был
=====
,
=====
а
=====
##ник
=====
и
=====
его
=====
##пруга
=====
очень
=====
,
=====
,
=====
когда
=====
к
=====
ним
=====
##сь
=====
мы
=====
с
=====
,
=====
в
=====
##ке
=====
стало
=====
##сем
=====
.
=====


{2: [['он',
   'человек',
   'отец',
   'не',
   'муж',
   '##ень',
   'сам',
   '##чик',
   'же',
   'сын']],
 3: [['очень',
   'довольно',
   'слишком',
   'достаточно',
   'весьма',
   'совершенно',
   'ещё',
   'много',
   'особенно',
   'самым']],
 7: [['«', '"', '...', ',', '-', '„', 'и', 'но', '»', "'"]],
 8: [['его',
   'сам',
   'их',
   'потому',
   'этот',
   'тот',
   'он',
   'тогда',
   'её',
   'также']],
 11: [[',', 'он', 'его', 'я', '-', '##чик', 'как', 'сам', '...', 'и']],
 12: [['жена',
   'мать',
   'отец',
   'дети',
   'голова',
   'муж',
   'дочь',
   'я',
   'у',
   'голос']],
 13: [['жена',
   'вторая',
   'первая',
   'же',
   'сестра',
   'новая',
   'дочь',
   'младший',
   'старший',
   'друг']],
 15: [['были',
   '-',
   'оказались',
   'была',
   'стали',
   'был',
   'не',
   '##ми',
   ',',
   'тоже']],
 16: [['были',
   'много',
   'очень',
   'с',
   'ещё',
   'слишком',
   'же',
   'не',
   'немного',
   'так']],
 19: [['и', 'Именно', 'И', 'именно', 

In [71]:
tokenizer.tokenize('были')

['были']

In [33]:
tokenizer.convert_ids_to_tokens(tmp)

['я', 'Я', '.', '我', 'I', ',', 'es', 'me', '-', 'je']

In [177]:
def middle_layer(tok_sent, verb, tokenizer, index, model, threshold, sent):
    # принимает на вход предложение, глагол, токенизатор, индекс, модель и threshold+sent; возвращает список форм глагола
    # на этом же слое должно происходить объединение результатов (есть ли гэппинг)
    # пока что всё делаем для одной формы глагола
    is_gapping = 0
    V_pos = []
    V_pos_res = ""
    cV_pos = "-1:-1"
    tok_verb = tokenizer.tokenize(verb)
    generated_words = bottom_layer(tok_sent, [tok_verb], tokenizer, index, model, threshold) # словарь
    print(generated_words)
    for key in generated_words.keys():
        assert len(generated_words[key]) == len(tok_verb)
        assert len(generated_words[key]) > 0
        gapping_flag = 1
        for i in range(len(tok_verb)):
            if tok_verb[i] not in generated_words[key][i]:
                gapping_flag = 0
        if (gapping_flag == 1):
            is_gapping = 1
            V_pos.append(key)
            if (cV_pos == "-1:-1"):
                tmp = sent.rfind(verb)
                cV_pos = f"{tmp}:{tmp + len(verb)}"
    for elem in V_pos: #TODO: реализовать быстрый поиск позиции слова
        tmp = sent.rfind(tok_sent[elem])
        V_pos_res += f"{tmp}:{tmp} "
    return is_gapping, cV_pos, V_pos_res

In [204]:
def top_layer(sent, tokenizer, model, sentence_tokenizer, threshold = 10):
    # на вход - предложение, токенизатор, модель, threshold, токенизатор для предложений (BasicTokenizer)
    sent = sent.replace('й', 'и').replace('ё', 'е').lower()
    tok_sent = tokenizer.tokenize(sent)
    tok_verb_sent = sentence_tokenizer.tokenize(sent)
    verb_list = [word for word in tok_verb_sent if check_verb_in_tags(word) == 'VERB']
    print(verb_list)
    for verb in verb_list:
        verb_pos = sent.rfind(verb)
        tok_prefix = tokenizer.tokenize(sent[:(verb_pos+len(verb))])
        index = len(tok_prefix) + 1
        print(index)
        is_gapping, cV_pos, V_pos = middle_layer(tok_sent, verb, tokenizer, index, model, threshold, sent)
        if (is_gapping == 1):
            return is_gapping, cV_pos, V_pos
    return 0, "-1:-1", "-1:-1"

In [179]:
from pytorch_pretrained_bert import BasicTokenizer

In [246]:
tmp = BasicTokenizer()
st = 'Для обжига садки изделий в печи создают дополнительный поток тепла, направленный перпендикулярно основному потоку'
wtmp = tmp.tokenize(st)
tst = tokenizer.tokenize(st)

In [247]:
top_layer(st, tokenizer, model, tmp)

['для', 'садки', 'создают', 'тепла', 'перпендикулярно']
2
##га
=====
INPUT_TO_BERT: ['для', 'об', '##жи', '##га', '[MASK]', 'сад', '##ки', 'изд', '##ели', '##и', 'в', 'п', '##ечи', 'со', '##зда', '##ют', 'доп', '##ол', '##ните', '##льны', '##и', 'по', '##ток', 'те', '##п', '##ла', ',', 'направлен', '##ны', '##и', 'пер', '##пен', '##дик', '##уля', '##рно', 'основному', 'по', '##ток', '##у']
##ки
=====
INPUT_TO_BERT: ['для', 'об', '##жи', '##га', 'сад', '##ки', '[MASK]', 'изд', '##ели', '##и', 'в', 'п', '##ечи', 'со', '##зда', '##ют', 'доп', '##ол', '##ните', '##льны', '##и', 'по', '##ток', 'те', '##п', '##ла', ',', 'направлен', '##ны', '##и', 'пер', '##пен', '##дик', '##уля', '##рно', 'основному', 'по', '##ток', '##у']
##и
=====
INPUT_TO_BERT: ['для', 'об', '##жи', '##га', 'сад', '##ки', 'изд', '##ели', '##и', '[MASK]', 'в', 'п', '##ечи', 'со', '##зда', '##ют', 'доп', '##ол', '##ните', '##льны', '##и', 'по', '##ток', 'те', '##п', '##ла', ',', 'направлен', '##ны', '##и', 'пер', '##пен', 

(1, '0:3', '107:107 ')

In [184]:
tokenizer.tokenize('фотографироваться')

['ф', '##ото', '##граф', '##ировать', '##ся']

In [182]:
middle_layer(tst, 'подтвердили', tokenizer, 17, model, 20, st)

##ые
=====
INPUT_TO_BERT: ['по', 'результатам', 'со', '##рев', '##нова', '##нии', 'че', '##тве', '##ро', 'спорт', '##с', '##менов', 'под', '##тве', '##рди', '##ли', 'з', '##елен', '##ые', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'по', '##яса', ',', 'шест', '##еро', '-', 'же', '##лт', '##ые', '.']
,
=====
INPUT_TO_BERT: ['по', 'результатам', 'со', '##рев', '##нова', '##нии', 'че', '##тве', '##ро', 'спорт', '##с', '##менов', 'под', '##тве', '##рди', '##ли', 'з', '##елен', '##ые', 'по', '##яса', ',', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'шест', '##еро', '-', 'же', '##лт', '##ые', '.']
-
=====
INPUT_TO_BERT: ['по', 'результатам', 'со', '##рев', '##нова', '##нии', 'че', '##тве', '##ро', 'спорт', '##с', '##менов', 'под', '##тве', '##рди', '##ли', 'з', '##елен', '##ые', 'по', '##яса', ',', 'шест', '##еро', '-', '[MASK]', '[MASK]', '[MASK]', '[MASK]', 'же', '##лт', '##ые', '.']
.
=====
INPUT_TO_BERT: ['по', 'результатам', 'со', '##рев', '##нова', '##нии', 'че', '##тве', '##ро', 'спорт', '##с

(0, '-1:-1', '')

In [130]:
from extra import check_verb_in_tags

In [131]:
ltmp = [word for word in wtmp if check_verb_in_tags(word) == 'VERB']

In [132]:
st = st.replace('й', 'и').replace('ё', 'е')
for elem in ltmp:
    print(st.rfind(elem))

13
50
92


In [133]:
tst = tokenizer.tokenize(st)

In [134]:
ltmp

['видел', 'был', 'боли']

In [135]:
for elem in ltmp:
    print(st[:(st.rfind(elem) + len(elem))])

Он как будто видел
Он как будто видел что-то в глубине, и взгляд его был
Он как будто видел что-то в глубине, и взгляд его был рассеянным, а выражение лица — полным боли


In [137]:
middle_layer(tst, 'был', tokenizer, 4, model, 10, st)

##л
=====
-
=====
то
=====
в
=====
,
=====
и
=====
##гляд
=====
его
=====
был
=====
,
=====
а
=====
##ражение
=====
[UNK]
=====
##ным
=====
.
=====


(1, (50, 53), '85:85 ')